In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/luis/uni/dl-vc/voxl3r


/Users/luis/uni/dl-vc/voxl3r/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig
from models.surface_net_baseline.util import project_points_to_images
from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [ ]:
# Init dataset and load for visualization
dataset_tdf = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="tdf",
    visualize=True,
)

dataset = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="tdf",
    visualize=True,
)

device = get_default_device()
transform = SceneDatasetTransformToTorch(target_device=device)

In [8]:
idx = dataset.get_index_from_scene("8b2c0938d6")
data = dataset[idx]

image_names, camera_params_list, _ = data["images"]
images, transformations, points, gt = transform.forward(data)
# and normalize images
images = images / 255.0

/Users/luis/uni/dl-vc/voxl3r/dataset.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luis/uni/dl-vc/voxl3r/dataset.py:302: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)


In [9]:
from dataset import plot_training_example

plot_training_example(dataset, idx)

Widget(value='<iframe src="http://localhost:60054/index.html?ui=P_0x2aecff740_1&reconnect=auto" class="pyvista…

In [8]:

# from dataset import plot_mask

# plot_mask(dataset, idx)

In [9]:
from dataset import plot_occupency_grid

plot_occupency_grid(dataset, idx)

Voxel Grid Size (L, W, H): (np.float64(2.0927280955686047), np.float64(2.004165828246644), np.float64(1.308528166040095))
Voxel Grid Center: [3.75472152 2.71583842 2.03600226]
Occupied Voxels: 7272.0


Widget(value='<iframe src="http://localhost:59905/index.html?ui=P_0x2a77dcaa0_1&reconnect=auto" class="pyvista…